## LOAD SOLUTION FILES

In [1]:
import os
import pickle
import pandas as pd
from tf_attention import *
from tf_helpers import *

### Test Encoder

In [14]:
# print(torch.cuda.is_available())
%run tf_helpers.py
import tensorflow as tf

size = 32
sequence_length = 625
input_dim = 9
output_dim = 3

path_encoded = path_encoder()
X = []
Y = []
network = []
for i in tqdm(range(size)) :
    file_name = f"Output/5by5_Data{i}"
    x, y = generate_xy(file_name, path_encoded)
    X.append(x)
    Y.append(y)

    # file = open(file_name, "rb")
    # stat = pickle.load(file)
    # file.close()
    # net = stat["data"]["network"]
    # nodes = stat["data"]["nodes"]
    # Graph = get_graphTensor(net, nodes)
    # network.append(Graph)
X = tf.stack(X, axis=0)
Y = tf.stack(Y, axis=0)
    

  0%|          | 0/32 [00:00<?, ?it/s]

In [3]:
network[0]

TensorShape([32, 625, 3])

In [16]:
a = pd.DataFrame(X[0].numpy())
a.head()
# a[a["1"] != 0]

,0,1,2,3,4,5,6
0,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09
1,1.000000e+00,1.000000e+00,1.000000e+00,8.908909e-02,6.733102e-01,6.365218e-01,2.233766e-01
2,-1.000000e+09,-1.000000e+09,-1.000000e+09,8.558559e-01,4.012132e-01,4.695652e-01,9.948052e-01
3,-1.000000e+09,-1.000000e+09,-1.000000e+09,1.471471e-01,3.734835e-01,3.721739e-01,1.705628e-01
4,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09,-1.000000e+09


In [5]:
in_mask = tf.cast(tf.not_equal(X, 0), dtype=tf.float32) # TensorShape([32, 625, 9])
X = X * in_mask # TensorShape([32, 625, 9])
out_mask = tf.cast(tf.not_equal(Y, 0), dtype=tf.float32)
Y = Y * out_mask
out_mask.shape

TensorShape([32, 625, 3])

In [17]:
%run tf_attention.py

encoder = Encoder(input_dim=7, d_model=512, N=6, heads=8, dropout=0.1)
encoder_output, attention_scores = encoder(X)
print(encoder_output.shape)

(32, 625, 3)


In [18]:
pd.DataFrame(encoder_output[0].numpy()).head()

,0,1,2
0,0.628605,0.574423,2.424313
1,1.050829,2.537739,1.043929
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000
4,0.628605,0.574423,2.424313


### Test Decoder

In [19]:
%run tf_attention.py
decoder = Decoder(output_dim=3, d_model=512, N=6, heads=8, dropout=0.1)
decoder_out, scores = decoder(Y, encoder_output)

In [20]:
pd.DataFrame(decoder_out[0].numpy()).head()

,0,1,2
0,-0.850624,-0.552353,1.402977
1,1.085323,0.241594,-1.326916
2,1.098596,0.220972,-1.319568
3,1.098595,0.220972,-1.319568
4,-0.850624,-0.552353,1.402977


### Test Transformer

In [21]:
%run tf_attention.py

transformer = Transformer(7,3,512,6,8,0.1)
transformer.eval()
out = transformer(X, Y)
pd.DataFrame(out[0].numpy()).head()

,0,1,2
0,0.000000,0.000000,0.000000
1,1.091591,-0.443280,-0.241297
2,1.065382,-0.431197,-0.264500
3,1.065382,-0.431197,-0.264500
4,0.000000,0.000000,0.000000


### Extract OD demand and path set (X and Y)
X: OD demand, graph (link feature), path, link-path adj \
Y: path flow

In [22]:
%run tf_attention.py
%run tf_helpers.py 

class Dataset:
    def __init__(self, size, standard_norm, start_from=0):
        self.path_encoded = path_encoder()  # Get path encode dictionary
        self.X = []
        self.Y = []

        for i in tqdm(range(size)):
            file_name = f"Output/5by5_Data{start_from+i}"
            x, y = generate_xy(file_name, self.path_encoded, standard_norm)
            self.X.append(x)
            self.Y.append(y)
        
        self.X = tf.stack(self.X, axis=0)
        self.Y = tf.stack(self.Y, axis=0)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

    def to_tf_dataset(self, batch_size):
        dataset = tf.data.Dataset.from_tensor_slices((self.X, self.Y))
        dataset = dataset.shuffle(buffer_size=len(self.X)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset 
    

In [23]:
# %run helpers.py
batch_size = 32
train_size = 1000
val_size = 500
standard_norm = 'normalize'

train_dataset = Dataset(train_size, standard_norm)
train_data_loader = train_dataset.to_tf_dataset(batch_size)

val_dataset = Dataset(val_size, standard_norm, start_from=train_size)
val_data_loader = val_dataset.to_tf_dataset(batch_size)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

### TRY TRAINING AND VALIDATING

In [24]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
%run tf_attention.py

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# device = tf.device('gpu' if tf.test.is_gpu_available() else 'cpu')
device = 'cpu'
input_dim = 7
output_dim = 3

model = Transformer(input_dim=input_dim, output_dim=output_dim, d_model=512, N=2, heads=8, dropout=0.1)

epochs = 100
gradient_accumulation_steps = 4
learning_rate = 0.001

loss_fn = MeanSquaredError()
optimizer = Adam(learning_rate=learning_rate)

train_losses = []
val_losses = []

# Training loop
with tqdm(total=epochs, unit="epoch") as pbar:
    for epoch in range(epochs):    
        # Training phase
        total_train_loss = 0
        for src, trg in train_data_loader:
            with tf.device(device):
                with tf.GradientTape() as tape:
                    output = model(src, trg)
                    loss = loss_fn(trg, output)
                
                # Backpropagate and update the model
                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                total_train_loss += loss.numpy()
        
        # Validation phase
        model.eval()
        total_val_loss = 0
        for src, trg in val_data_loader:
            # Move the batch to the device
            with tf.device(device):
                # Forward pass
                output = model(src, trg)
                loss = loss_fn(trg, output)
                total_val_loss += loss.numpy()
            
        pbar.update(1)
        pbar.set_description(f"Train Loss: {total_train_loss / len(train_data_loader):.4f}, Val Loss: {total_val_loss / len(val_data_loader):.4f}")
        
        train_losses.append(total_train_loss / len(train_data_loader))
        val_losses.append(total_val_loss / len(val_data_loader))
        
        # Print the training and validation losses
        print(f"Epoch: {epoch+1} - Train Loss: {total_train_loss/len(train_data_loader):.4f}, Val Loss: {total_val_loss/len(val_data_loader):.4f}")


  0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 1 - Train Loss: 120253.8623, Val Loss: 54295.6248
Epoch: 2 - Train Loss: 121519.5427, Val Loss: 54089.1565


In [ ]:
def plot_loss(train_losses, val_losses, epochs):
    plt.figure(figsize=(12, 6))
    plt.plot(range(1, epochs+1), train_losses, label='Training Loss')
    plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
train_losses = [0.8, 0.6, 0.5, 0.4, 0.3]
val_losses = [0.9, 0.7, 0.6, 0.5, 0.4]
epochs = 5

plot_loss(train_losses, val_losses, epochs)